In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import (
    StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
)
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, when, isnull

# %%
# Configurar SparkSession
spark = SparkSession.builder \
    .appName("SECOP_FeatureEngineering") \
    .master("local[*]") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

In [7]:
# Cargar datos
df = spark.read.parquet("/opt/spark-data/processed/secop_eda.parquet")
print(f"Registros cargados: {df.count():,}")

# %%
# Explorar columnas disponibles
print("Columnas disponibles:")
for col_name in df.columns:
    print(f"  - {col_name}")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/opt/spark-data/processed/secop_eda.parquet.

In [ ]:
# Seleccionar features para el modelo
# Variables categóricas
categorical_cols = ["departamento", "tipo_de_contrato", "estado_contrato"]

# Variables numéricas
numeric_cols = ["plazo_de_ejec_del_contrato", "valor_del_contrato_num"]

# Verificar qué columnas existen
available_cat = [c for c in categorical_cols if c in df.columns]
available_num = [c for c in numeric_cols if c in df.columns]

print(f"Categóricas disponibles: {available_cat}")
print(f"Numéricas disponibles: {available_num}")

In [8]:
# %%
# Limpiar datos: eliminar nulos
df_clean = df.dropna(subset=available_cat + available_num)
print(f"Registros después de limpiar nulos: {df_clean.count():,}")

NameError: name 'df' is not defined

In [9]:
# Convierte strings a índices numéricos
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_idx", handleInvalid="keep")
    for col in available_cat
]

NameError: name 'available_cat' is not defined

In [10]:
print("StringIndexers creados:")
for idx in indexers:
    print(f"  - {idx.getInputCol()} -> {idx.getOutputCol()}")

# %%

StringIndexers creados:


NameError: name 'indexers' is not defined

In [11]:
# PASO 2: OneHotEncoder para generar variables dummy
encoders = [
    OneHotEncoder(inputCol=col + "_idx", outputCol=col + "_vec")
    for col in available_cat
]

NameError: name 'available_cat' is not defined

In [ ]:
print("\nOneHotEncoders creados:")
for enc in encoders:
    print(f"  - {enc.getInputCol()} -> {enc.getOutputCol()}")

In [ ]:
# PASO 3: VectorAssembler para combinar todas las features
# Combinamos: features numéricas + features categóricas codificadas
feature_cols = available_num + [col + "_vec" for col in available_cat]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features_raw"
)

print(f"\nVectorAssembler combinará: {feature_cols}")

In [5]:
# PASO 4: Construir Pipeline
# Pipeline = secuencia de transformaciones
pipeline_stages = indexers + encoders + [assembler]

pipeline = Pipeline(stages=pipeline_stages)

print(f"\nPipeline con {len(pipeline_stages)} stages:")
for i, stage in enumerate(pipeline_stages):
    print(f"  Stage {i+1}: {type(stage).__name__}")

NameError: name 'indexers' is not defined

In [ ]:
# PASO 5: Entrenar el pipeline (fit)
# Nota: StringIndexer y OneHotEncoder necesitan "aprender" del dataset
print("\nEntrenando pipeline...")
pipeline_model = pipeline.fit(df_clean)
print("Pipeline entrenado exitosamente")

In [ ]:

# %%
# PASO 6: Aplicar transformaciones (transform)
df_transformed = pipeline_model.transform(df_clean)

print("\nTransformación completada")
print(f"Columnas después de transformar: {len(df_transformed.columns)}")


In [ ]:
df_transformed.select("features_raw").printSchema()

In [ ]:
df_transformed.select("features_raw").show()

In [ ]:
# Ver dimensión del vector de features
sample_features = df_transformed.select("features_raw").first()[0]
print(f"Dimensión del vector de features: {len(sample_features)}")

# %%
# Mostrar ejemplo de transformación
df_transformed.select(
    available_cat[0] if available_cat else "id",
    available_cat[0] + "_idx" if available_cat else "id",
    available_cat[0] + "_vec" if available_cat else "id",
    "features_raw"
).show(5, truncate=True)

In [ ]:
# %%
# Guardar pipeline entrenado
pipeline_path = "/opt/spark-data/processed/feature_pipeline"
pipeline_model.save(pipeline_path)
print(f"\nPipeline guardado en: {pipeline_path}")

# %%
# Guardar dataset transformado
output_path = "/opt/spark-data/processed/secop_features.parquet"
df_transformed.write.mode("overwrite").parquet(output_path)
print(f"Dataset transformado guardado en: {output_path}")